# Basics

Let's first take a look at what's inside the ``ib_async`` package:

In [1]:
import ib_async

print(ib_async.__all__)
–


['Event', 'util', 'Client', 'Bag', 'Bond', 'CFD', 'ComboLeg', 'Commodity', 'ContFuture', 'Contract', 'ContractDescription', 'ContractDetails', 'Crypto', 'DeltaNeutralContract', 'Forex', 'Future', 'FuturesOption', 'Index', 'MutualFund', 'Option', 'ScanData', 'Stock', 'TagValue', 'Warrant', 'FlexError', 'FlexReport', 'IB', 'IBC', 'Watchdog', 'AccountValue', 'BarData', 'BarDataList', 'CommissionReport', 'ConnectionStats', 'DOMLevel', 'DepthMktDataDescription', 'Dividends', 'Execution', 'ExecutionFilter', 'FamilyCode', 'Fill', 'FundamentalRatios', 'HistogramData', 'HistoricalNews', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'HistoricalSchedule', 'HistoricalSession', 'MktDepthData', 'NewsArticle', 'NewsBulletin', 'NewsProvider', 'NewsTick', 'OptionChain', 'OptionComputation', 'PnL', 'PnLSingle', 'PortfolioItem', 'Position', 'PriceIncrement', 'RealTimeBar', 'RealTimeBarList', 'ScanDataList', 'ScannerSubscription', 'SmartComponent', 'SoftDollarTier', 'TickAttrib', 'TickAt

### Importing
The following two lines are used at the top of all notebooks. The first line imports everything and the second
starts an event loop to keep the notebook live updated:

In [3]:
from ib_async import *

util.startLoop()


*Note that startLoop() only works in notebooks, not in regular Python programs.*

### Connecting
The main player of the whole package is the "IB" class. Let's create an IB instance and connect to a running TWS/IBG application:

In [4]:
ib = IB()
ib.connect("127.0.0.1", 4001, clientId=1101)


<IB connected to 127.0.0.1:4001 clientId=1101>

If the connection failed, then verify that the application has the API port enabled and double-check the hostname and port. For IB Gateway the default port is 4002. Make sure the clientId is not already in use.

If the connection succeeded, then ib will be synchronized with TWS/IBG. The "current state" is now available via methods such as ib.positions(), ib.trades(), ib.openTrades(), ib.accountValues() or ib.tickers(). Let's list the current positions:

In [5]:
util.df(ib.accountValues(account="U10394496"))


,account,tag,value,currency,modelCode
0,U10394496,AccountCode,U10394496,,
1,U10394496,AccountReady,true,,
2,U10394496,AccountType,INDIVIDUAL,,
3,U10394496,ColumnPrio-C,2,,
4,U10394496,ColumnPrio-P,5,,
...,...,...,...,...,...
177,U10394496,TotalCashBalance,215752.36,USD,
178,U10394496,UnrealizedPnL,-343.16,BASE,
179,U10394496,UnrealizedPnL,-343.16,USD,
180,U10394496,WarrantValue,0.00,BASE,


In [8]:
ib.positions()


[Position(account='U2340948', contract=Stock(conId=4762, symbol='BA', exchange='NYSE', currency='USD', localSymbol='BA', tradingClass='BA'), position=254.0, avgCost=203.21112205),
 Position(account='U10394496', contract=Option(conId=637079300, symbol='HYG', lastTradeDateOrContractMonth='20240621', strike=73.0, right='P', multiplier='100', currency='USD', localSymbol='HYG   240621P00073000', tradingClass='HYG'), position=50.0, avgCost=12.44685),
 Position(account='U10394496', contract=Option(conId=682674010, symbol='AAPL', lastTradeDateOrContractMonth='20241018', strike=150.0, right='P', multiplier='100', currency='USD', localSymbol='AAPL  241018P00150000', tradingClass='AAPL'), position=2.0, avgCost=244.69685),
 Position(account='U10394496', contract=Option(conId=657576926, symbol='QQQ', lastTradeDateOrContractMonth='20240930', strike=376.78, right='P', multiplier='100', currency='USD', localSymbol='QQQ   240930P00376780', tradingClass='QQQ'), position=1.0, avgCost=501.03685),
 Positio

In [12]:
util.tree(fills)


[[{'contract': {'Future': {'secType': 'FUT',
     'conId': 620730920,
     'symbol': 'NQ',
     'lastTradeDateOrContractMonth': '20240621',
     'right': '?',
     'multiplier': '20',
     'exchange': 'CME',
     'currency': 'USD',
     'localSymbol': 'NQM4',
     'tradingClass': 'NQ'}},
   'execution': {'Execution': {'execId': '0000d94c.66341896.01.01',
     'time': '2024-05-03T07:42:05+00:00',
     'acctNumber': 'U10394496',
     'exchange': 'CME',
     'side': 'SLD',
     'shares': 1.0,
     'price': 17726.25,
     'permId': 1872456553,
     'clientId': 22,
     'orderId': 209,
     'cumQty': 1.0,
     'avgPrice': 17726.25,
     'lastLiquidity': 1}},
   'commissionReport': {'CommissionReport': {'execId': '0000d94c.66341896.01.01',
     'commission': 2.2,
     'currency': 'USD'}},
   'time': '2024-05-03T07:42:05+00:00'}]]

In [6]:
fills = [t.fills for t in ib.trades() if t.fills != []]
executions = [f[0].execution for f in fills]
util.df(executions)


,execId,time,acctNumber,exchange,side,shares,price,permId,clientId,orderId,liquidation,cumQty,avgPrice,orderRef,evRule,evMultiplier,modelCode,lastLiquidity,pendingPriceRevision
0,0000d94c.66341896.01.01,2024-05-03 07:42:05+00:00,U10394496,CME,SLD,1.0,17726.25,1872456553,22,209,0,1.0,17726.25,,,0.0,,1,False


In [11]:
fills[0]


[Fill(contract=Future(conId=620730920, symbol='NQ', lastTradeDateOrContractMonth='20240621', right='?', multiplier='20', exchange='CME', currency='USD', localSymbol='NQM4', tradingClass='NQ'), execution=Execution(execId='0000d94c.66341896.01.01', time=datetime.datetime(2024, 5, 3, 7, 42, 5, tzinfo=datetime.timezone.utc), acctNumber='U10394496', exchange='CME', side='SLD', shares=1.0, price=17726.25, permId=1872456553, clientId=22, orderId=209, liquidation=0, cumQty=1.0, avgPrice=17726.25, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=1, pendingPriceRevision=False), commissionReport=CommissionReport(execId='0000d94c.66341896.01.01', commission=2.2, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2024, 5, 3, 7, 42, 5, tzinfo=datetime.timezone.utc))]

In [ ]:
ib.positions()


In [ ]:
ib.accountValues()


Or filter the account values to get the liquidation value:

In [ ]:
ib.accountValues(account="U10394496")


In [ ]:
[
    v
    for v in ib.accountValues()
    if v.tag == "NetLiquidationByCurrency" and v.currency == "BASE"
]


The "current state" will automatically be kept in sync with TWS/IBG. So an order fill will be added as soon as it is reported, or account values will be updated as soon as they change in TWS.

### Contracts

Contracts can be specified in different ways:
* The ibapi way, by creating an empty Contract object and setting its attributes one by one;
* By using Contract and giving the attributes as keyword argument;
* By using the specialized Stock, Option, Future, Forex, Index, CFD, Commodity,
  Bond, FuturesOption, MutualFund or Warrant contracts.

Some examples:

In [1]:
Contract(conId=270639)
Stock("AMD", "SMART", "USD")
Stock("INTC", "SMART", "USD", primaryExchange="NASDAQ")
Forex("EURUSD")
CFD("IBUS30")
Future("ES", "20180921", "GLOBEX")
Option("SPY", "20170721", 240, "C", "SMART")
Bond(secIdType="ISIN", secId="US03076KAA60")


NameError: name 'Contract' is not defined

### Sending a request

The IB class has nearly all request methods that the IB API offers. The methods that return a result will block until finished and then return the result. Take for example reqContractDetails:

In [ ]:
contract = Stock("TSLA", "SMART", "USD")
ib.reqContractDetails(contract)


### Current state vs request

Doing a request involves network traffic going up and down and can take considerable time. The current state on the other hand is always immediately available. So it is preferable to use the current state methods over requests. For example, use ``ib.openOrders()`` in preference over ``ib.reqOpenOrders()``, or ``ib.positions()`` over ``ib.reqPositions()``, etc:

In [ ]:
%time l = ib.positions()


In [ ]:
%time l = ib.reqPositions()


### Logging

The following will put log messages of INFO and higher level under the current active cell:

In [ ]:
util.logToConsole()


To see all debug messages (including network traffic):

In [ ]:
import logging

util.logToConsole(logging.DEBUG)


### Disconnecting

The following will disconnect ``ib`` and clear all its state:

In [ ]:
ib.disconnect()
